# Resources:

https://towardsdatascience.com/cifar-100-transfer-learning-using-efficientnet-ed3ed7b89af2

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf 
from tensorflow.keras.applications.efficientnet import EfficientNetB7  
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping 
from glob import glob
import pickle
from keras.models import Sequential, load_model
from keras.layers import Dropout, Dense, GlobalAveragePooling2D

In [3]:
from sklearn.model_selection import StratifiedShuffleSplit
import cv2
import albumentations as albu
from skimage.transform import resize
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import rcParams
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
import keras
from keras.models import Sequential, load_model
from keras.layers import Dropout, Dense, GlobalAveragePooling2D
from keras.optimizers import Adam

In [4]:
v=EfficientNetB7(
    include_top=False, weights='imagenet', input_tensor=None,
    input_shape=[600, 600] + [3], pooling=None, classes=9,
    classifier_activation='softmax'
)

258080768/258076736 [==============================] - 1s 0us/step


In [5]:
epochs = 2
height = 224
width = 224
channels = 3
input_shape = (height, width, channels)
#input_shape = [600, 600] + [3]
n_classes = 9

efnb7 = EfficientNetB7(weights='imagenet', include_top=False, input_shape=input_shape, classes=n_classes)
for layer in efnb7.layers:
        layer.trainable = False

model = Sequential()
model.add(efnb7)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))

input_shape_new = (None, height, width, channels)
model.build(input_shape_new)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper (ModuleWrappe (None, 7, 7, 2560)        64097687  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2560)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2560)              0         
_________________________________________________________________
dense (Dense)                (None, 9)                 23049     
Total params: 64,120,736
Trainable params: 23,049
Non-trainable params: 64,097,687
_________________________________________________________________


In [6]:
optimizer = Adam(lr=0.0001)

#early stopping to monitor the validation loss and avoid overfitting
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)

#reducing learning rate on plateau
rlrop = ReduceLROnPlateau(monitor='val_loss', mode='min', patience= 5, factor= 0.5, min_lr= 1e-6, verbose=1)
#model compiling
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [10]:
train_datagen = ImageDataGenerator(zoom_range = 0.2, 
                                   rescale = 1./255,
                                   shear_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split=0.2)

In [12]:
training_set = train_datagen.flow_from_directory("/content/drive/MyDrive/DL-Project/Colab/dataset",
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 subset='training')

Found 6670 images belonging to 9 classes.


In [13]:
valid_data_generator = train_datagen.flow_from_directory("/content/drive/MyDrive/DL-Project/Colab/dataset",
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 subset='validation')

Found 1665 images belonging to 9 classes.


In [14]:
epochs = 10

In [ ]:
model_history = model.fit_generator(training_set, validation_data = valid_data_generator, callbacks = [early_stop, rlrop],verbose = 1, epochs = epochs)

#saving the trained model weights as data file in .h5 format
model.save_weights("/content/drive/MyDrive/DL-Project/Colab/Saved Models/eb7/efficientnetb7_weights.h5")

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
209/209 [==============================] - 5473s 26s/step - loss: 2.2205 - accuracy: 0.1197 - val_loss: 2.1734 - val_accuracy: 0.1465
Epoch 2/10
209/209 [==============================] - 3564s 17s/step - loss: 2.2143 - accuracy: 0.1206 - val_loss: 2.1761 - val_accuracy: 0.1465
Epoch 3/10
209/209 [==============================] - 4143s 20s/step - loss: 2.2145 - accuracy: 0.1328 - val_loss: 2.1747 - val_accuracy: 0.0961
Epoch 4/10
209/209 [==============================] - 3808s 18s/step - loss: 2.2067 - accuracy: 0.1241 - val_loss: 2.1738 - val_accuracy: 0.1465
Epoch 5/10
209/209 [==============================] - 3338s 16s/step - loss: 2.2099 - accuracy: 0.1284 - val_loss: 2.1721 - val_accuracy: 0.1465
Epoch 6/10
209/209 [==============================] - 3574s 17s/step - loss: 2.2083 - accuracy: 0.1249 - val_loss: 2.1730 - val_accuracy: 0.1465
Epoch 7/10
209/209 [==============================] - 3151s 15s/step - loss: 2.2043 - accuracy: 0.1380 - val_loss: 2.1721 - val_ac

In [ ]:
r = model_history

In [ ]:
#accuracies
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('/content/drive/MyDrive/DL-Project/Colab/Saved Models/eb7/eb7_acc')

# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('/content/drive/MyDrive/DL-Project/Colab/Saved Models/eb7/eb7_loss')


# model.save('/content/drive/MyDrive/DL-Project/Colab/Saved Models/eb7/eb7_model.h5')
model.save('/content/drive/MyDrive/DL-Project/Colab/Saved Models/eb7/eb7_Model')      
model.save_weights('/content/drive/MyDrive/DL-Project/Colab/Saved Models/eb7/eb7_weights_model_1.h5')  

# with open('/content/drive/MyDrive/DL-Project/Colab/Saved Models/eb7/eb7_model.json', 'w') as f:
#     f.write(model.to_json())
 
file = open( "/content/drive/MyDrive/DL-Project/Colab/Saved Models/eb7/eb7_model_acc",'wb') 
pickle.dump(list(r.history['accuracy']), file) 
file.close()

file = open( "/content/drive/MyDrive/DL-Project/Colab/Saved Models/eb7/eb7_model_loss",'wb') 
pickle.dump(list(r.history['loss']), file) 
file.close()  

file = open( "/content/drive/MyDrive/DL-Project/Colab/Saved Models/eb7/eb7_model_val_acc",'wb') 
pickle.dump(list(r.history['val_accuracy']), file) 
file.close() 

file = open( "/content/drive/MyDrive/DL-Project/Colab/Saved Models/eb7/eb7_model_val_loss",'wb') 
pickle.dump(list(r.history['val_loss']), file) 
file.close() 